<h1>Data science Capstone project</h1>

<h2>1. Business understanding</h2>

<p>Suppose a pub chain: "Pin't" is planning to open a new pub in Dublin. We need to identify the neighbourhood which will likely be a success for the pub. The chain has several pubs in London. Each with a different ROI (Revenue on investment).</p>
<h5>Available data:</h5> 
    <li>Geolocation of existing pubs and potential neighbourhoods for new pub</li>
    <li>ROI of existing pubs</li>
<h5>Problem:</h5>
<p>
How can we identify a neighborhood, based on its location and the chain’s history and experience in other cities?</p>
<h5>Approach:</h5>
<p>Our approach is to use Foursquare location data and a clustering algorithm in order to cluster the potential neighbourhoods and the neighbourhoods where the chain has already established a pub, according to the category of venues close to each neighbourhood. And then choose the neighbourhood that belongs to the cluster with the most success rate. If there is a conflict: two potential neighbourhoods belong to most successful cluster we’ll be using other metrics: choose the neighbourhood with less pubs (for example)</p>
<h1>2. Data description:</h1>
<li>Categories of the venues in each eighbourhood venues (Foursquare API): It will help us group similar neighbourhoods.</li> 

<li>Existing pubs data: It will help us identify the most successful cluster. <a href="https://docs.google.com/spreadsheets/d/1UrTzTew7otS3l2AZS3aDeT2LqeLolHafxf1-3wIig08/edit?usp=sharing">https://docs.google.com/spreadsheets/d/1UrTzTew7otS3l2AZS3aDeT2LqeLolHafxf1-3wIig08/edit?usp=sharing</a></li>

<li>Potential neighbourhoods in Dublin: <a href="https://docs.google.com/spreadsheets/d/1QNCQQIdCHGsi6Re8IfioJaHm8ab5C11RQTGTXm-756M/edit?usp=sharing">https://docs.google.com/spreadsheets/d/1QNCQQIdCHGsi6Re8IfioJaHm8ab5C11RQTGTXm-756M/edit?usp=sharing</a></li>


<h3>2.1 Existing pubs data:</h3>

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [7]:
import pandas as pd
import requests
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
london_pubs=pd.read_csv('existing_pubs.csv')
london_pubs.head()

,Neighbourhood,ROI
0,Barking and Dagenham,0.20
1,Barnet,0.76
2,Bexley,0.38
3,Brent,0.74
4,Bromley,0.29


In [8]:
def get_location(row,city):
    Borough=row['Neighbourhood']
    a=Borough+','+city
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(a)
    latitude = location.latitude
    longitude = location.longitude
    row['latitude']=latitude
    row['longitude']=longitude
    return row

In [9]:
london_pubs=london_pubs.apply(lambda row: get_location(row,'London'), axis=1)


In [10]:
london_pubs.head()

,Neighbourhood,ROI,latitude,longitude
0,Barking and Dagenham,0.20,51.554117,0.150504
1,Barnet,0.76,51.648784,-0.172913
2,Bexley,0.38,51.441679,0.150488
3,Brent,0.74,51.563826,-0.275760
4,Bromley,0.29,51.402805,0.014814


<h5><li>Visualizing existing pubs in London</li></h5>

In [11]:

address = 'London, England'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [12]:

map_london = folium.Map(location=[latitude,longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(london_pubs['latitude'],london_pubs['longitude'], london_pubs['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

<h5><li>Getting venues near existing pubs in London</li></h5>

In [13]:
CLIENT_ID = 'UT1GTTMAXJOA11RRWCSGYTF20AU1CHM4AQODWJLJVWLKNMWP' # your Foursquare ID
CLIENT_SECRET = 'EEAB2I5ARKJFIL4DAEJWIMBDR5XH1NRKD5DVFSTBHPG05KZ1' # your Foursquare Secret
VERSION = '20180605'
LIMIT=30
radius=5000
def explore(neighborhood_latitude, neighborhood_longitude):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
    results = requests.get(url).json()
    return results

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        a = explore(lat,lng)["response"]
        
        results=a['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
venues_in_london = getNearbyVenues(names=london_pubs['Neighbourhood'],
                                   latitudes=london_pubs['latitude'],
                                   longitudes=london_pubs['longitude']
                                  )
venues_in_london.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Barking and Dagenham,30,30,30,30,30,30
Barnet,30,30,30,30,30,30
Bexley,30,30,30,30,30,30
Brent,30,30,30,30,30,30
Bromley,30,30,30,30,30,30
Camden,30,30,30,30,30,30
Croydon,30,30,30,30,30,30
Ealing,30,30,30,30,30,30
Enfield,30,30,30,30,30,30


In [16]:
london_onehot = pd.get_dummies(venues_in_london[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighbourhood'] = venues_in_london['Neighbourhood'] 
# move neighborhood column to the first column
index=london_onehot.columns.get_loc("Neighbourhood")
fixed_columns = [london_onehot.columns[index]] + list(london_onehot.columns[:index])+list(london_onehot.columns[index+1:])
london_onehot = london_onehot[fixed_columns]
london_onehot.head(3)


,Neighbourhood,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bakery,Bar,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Canal,Canal Lock,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Fountain,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Noodle House,Observatory,Opera House,Optical Shop,Palace,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Record Shop,Recreation Center,Restaurant,Rugby Stadium,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Social Club,South Indian Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
0,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<h5><li>Grouping the venues by neighborhoods and displaying the frequency of each category</li></h5>


In [17]:

london_grouped = london_onehot.groupby('Neighbourhood').mean().reset_index()
london_grouped

,Neighbourhood,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Bakery,Bar,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Canal,Canal Lock,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Donut Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Fountain,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Noodle House,Observatory,Opera House,Optical Shop,Palace,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pizza Place,Planetarium,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Record Shop,Recreation Center,Restaurant,Rugby Stadium,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Social Club,South Indian Restaurant,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
0,Barking and Dagenham,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.233333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Barnet,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000

<h5><li>Sorting the venues near existing pubs in London</li></h5>

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = london_grouped['Neighbourhood']

for ind in np.arange(london_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,Park,Coffee Shop,Furniture / Home Store,Café,Portuguese Restaurant,Italian Restaurant,Movie Theater,Multiplex,Supermarket,Clothing Store
1,Barnet,Café,Park,Coffee Shop,Turkish Restaurant,Pub,Supermarket,Dessert Shop,Theater,Movie Theater,Breakfast Spot
2,Bexley,Pub,Italian Restaurant,Park,Café,American Restaurant,Museum,Clothing Store,Greek Restaurant,Performing Arts Venue,English Restaurant
3,Brent,Indian Restaurant,Clothing Store,Hotel,Portuguese Restaurant,Sandwich Place,Middle Eastern Restaurant,Latin American Restaurant,Sporting Goods Shop,Sports Club,Stadium
4,Bromley,Pub,Park,Coffee Shop,Gym / Fitness Center,Pizza Place,Cocktail Bar,Burger Joint,Furniture / Home Store,Portuguese Restaurant,Bookstore


<h3>2.1 Potential pubs data:</h3>

In [20]:
potential_pubs=pd.read_csv('potential_pubs.csv')
potential_pubs

,Neighbourhood
0,St. Stephen’s Green
1,Temple Bar
2,Christchurch
3,Ranelagh and Rathmines
4,Ballsbridge and Donnybrook
5,Drumcondra
6,Malahide
7,Dalkey


In [21]:
potential_pubs=potential_pubs.apply(lambda row: get_location(row,'Dublin'), axis=1)

In [22]:
potential_pubs

,Neighbourhood,latitude,longitude
0,St. Stephen’s Green,53.337990,-6.259073
1,Temple Bar,53.345496,-6.263114
2,Christchurch,53.342689,-6.272784
3,Ranelagh and Rathmines,53.325218,-6.255050
4,Ballsbridge and Donnybrook,53.335711,-6.245229
5,Drumcondra,53.372525,-6.249515
6,Malahide,53.450840,-6.153670
7,Dalkey,53.275607,-6.103188


<h5><li>Visualizing potential pubs in Dublin</li></h5>

In [24]:

address = 'Dublin, Ireland'

geolocator = Nominatim(user_agent="ny_explorer")
d_location = geolocator.geocode(address)
d_latitude = d_location.latitude
d_longitude = d_location.longitude
print('The geograpical coordinate of Dublin are {}, {}.'.format(d_latitude, d_longitude))


The geograpical coordinate of Dublin are 53.3497645, -6.2602732.


In [25]:
map_dublin = folium.Map(location=[d_latitude,d_longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(potential_pubs['latitude'],potential_pubs['longitude'], potential_pubs['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dublin)  
    
map_dublin

<h5><li>Getting venues near potential pubs in Dublin</li></h5>

In [26]:
venues_in_dublin = getNearbyVenues(names=potential_pubs['Neighbourhood'],
                                   latitudes=potential_pubs['latitude'],
                                   longitudes=potential_pubs['longitude']
                                  )
venues_in_dublin.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Ballsbridge and Donnybrook,30,30,30,30,30,30
Christchurch,30,30,30,30,30,30
Dalkey,30,30,30,30,30,30
Drumcondra,30,30,30,30,30,30
Malahide,30,30,30,30,30,30
Ranelagh and Rathmines,30,30,30,30,30,30
St. Stephen’s Green,30,30,30,30,30,30
Temple Bar,30,30,30,30,30,30


In [27]:
dublin_onehot = pd.get_dummies(venues_in_dublin[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dublin_onehot['Neighbourhood'] = venues_in_dublin['Neighbourhood'] 
# move neighborhood column to the first column
index=dublin_onehot.columns.get_loc("Neighbourhood")
fixed_columns = [dublin_onehot.columns[index]] + list(dublin_onehot.columns[:index])+list(dublin_onehot.columns[index+1:])
dublin_onehot = dublin_onehot[fixed_columns]
dublin_onehot.head(3)



,Neighbourhood,American Restaurant,Art Museum,Bakery,Bar,Beach,Beer Bar,Bistro,Bookstore,Botanical Garden,Burger Joint,Café,Canal,Castle,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Deli / Bodega,Dessert Shop,Diner,Discount Store,Donut Shop,Falafel Restaurant,Farmers Market,Fish & Chips Shop,Fish Market,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Gourmet Shop,Grocery Store,Gym,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Korean Restaurant,Liquor Store,Lounge,Mexican Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Museum,Music Venue,Outdoor Sculpture,Park,Pizza Place,Plaza,Pub,Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Yoga Studio
0,St. Stephen’s Green,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,St. Stephen’s Green,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,St. Stephen’s Green,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [28]:
dublin_grouped = dublin_onehot.groupby('Neighbourhood').mean().reset_index()
dublin_grouped

,Neighbourhood,American Restaurant,Art Museum,Bakery,Bar,Beach,Beer Bar,Bistro,Bookstore,Botanical Garden,Burger Joint,Café,Canal,Castle,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Deli / Bodega,Dessert Shop,Diner,Discount Store,Donut Shop,Falafel Restaurant,Farmers Market,Fish & Chips Shop,Fish Market,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Gourmet Shop,Grocery Store,Gym,Historic Site,History Museum,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Korean Restaurant,Liquor Store,Lounge,Mexican Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Museum,Music Venue,Outdoor Sculpture,Park,Pizza Place,Plaza,Pub,Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Yoga Studio
0,Ballsbridge and Donnybrook,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.200000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.100000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.100000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.100000,0.033333,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Christchurch,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000
2,Dalkey,0.000000,0.000000,0.000000,0.033333,0.133333,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.066667,0.066667,0.000000,0.066667,0.033333,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Drumcondra,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.133333,0.000000,0.033333,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.033333,0.000000,0.066667,0.066667,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Malahide,0.033333,0.000000,0.000000,0.033333,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000

<h5><li>Sorting venues near potential pubs</li></h5>

In [30]:
# create a new dataframe
potential_neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
potential_neighbourhoods_venues_sorted['Neighbourhood'] = dublin_grouped['Neighbourhood']

for ind in np.arange(dublin_grouped.shape[0]):
    potential_neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dublin_grouped.iloc[ind, :], num_top_venues)

potential_neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ballsbridge and Donnybrook,Café,Hotel,Coffee Shop,Park,Gym,Liquor Store,Lounge,Concert Hall,Cocktail Bar,Outdoor Sculpture
1,Christchurch,Café,Coffee Shop,Pub,Cocktail Bar,Burger Joint,Irish Pub,Indie Movie Theater,Music Venue,Ice Cream Shop,Park
2,Dalkey,Beach,Restaurant,Steakhouse,Park,Pub,Italian Restaurant,Scenic Lookout,Seafood Restaurant,Coffee Shop,Bistro
3,Drumcondra,Coffee Shop,Café,Restaurant,Pub,Gastropub,Korean Restaurant,Deli / Bodega,Mexican Restaurant,Clothing Store,Discount Store
4,Malahide,Café,Italian Restaurant,Beach,Garden,Indian Restaurant,Hotel,Gourmet Shop,Gift Shop,Garden Center,Gym
